In [1]:
from transformers import LEDTokenizerFast

import pandas as pd
import numpy as np
from tqdm import tqdm

In [9]:
tokenizer = LEDTokenizerFast.from_pretrained("allenai/led-base-16384")
df = pd.read_csv('train-old.csv') # type: pd.DataFrame
df["box_center"] = None
df["box_length"] = None

In [10]:
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,box_center,box_length
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,None,None
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,None,None
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,None,None
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,None,None
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,None,None
...,...,...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...,None,None
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...,None,None
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838,None,None
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...,None,None


In [11]:
for i in tqdm(range(len(df))):
  id = df.iloc[i]["id"]
  discourse_start = int(df.iloc[i]["discourse_start"])
  discourse_end = int(df.iloc[i]["discourse_end"])

  with open(f'train/{id}.txt', 'r') as f:
    text = f.read()

  char_array = np.zeros( (len(text)) )
  char_array[discourse_start : discourse_end] = 1

  encoding = tokenizer(text, padding='max_length', 
                      truncation=True, return_offsets_mapping=True)
  map_token_to_char = encoding['offset_mapping']

  len_sequence = len(map_token_to_char)

  token_array = np.zeros(len_sequence)
  for k in range(len(token_array)):
    token_array[k] = char_array[ map_token_to_char[k][0] - 1]

  pred = token_array.nonzero()[0]

  center = np.mean(pred) / len_sequence
  length = pred.shape[0] / len_sequence

  df.at[i, "center"] = center
  df.at[i, "length"] = length

  4%|▍         | 5536/144293 [01:14<41:31, 55.68it/s]  C:\Users\zumo0\anaconda3\envs\nlp\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\zumo0\anaconda3\envs\nlp\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 144293/144293 [40:28<00:00, 59.41it/s] 


In [12]:
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,box_center,box_length,center,length
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,None,None,0.001740,0.002930
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,None,None,0.003723,0.000916
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,None,None,0.004791,0.001099
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,None,None,0.007568,0.004333
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,None,None,0.010620,0.001648
...,...,...,...,...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...,None,None,0.032562,0.012817
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...,None,None,0.048187,0.017700
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838,None,None,0.057434,0.000793
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...,None,None,0.060181,0.004700


In [13]:
df.to_csv("train.csv")

In [19]:
df[df['center'].isna()]

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,box_center,box_length,center,length
5540,BB12314FD64A,1.622478e+12,301.0,307.0,safety,Claim,Claim 1,51,None,None,NaN,0.0
9299,36B65EF457C1,1.622757e+12,152.0,158.0,diesel,Claim,Claim 2,28,None,None,NaN,0.0
12323,A303B199A496,1.623082e+12,82.0,91.0,pollution,Claim,Claim 1,13,None,None,NaN,0.0
14467,34C979F3ABAA,1.622762e+12,96.0,106.0,protection,Claim,Claim 2,17,None,None,NaN,0.0
14468,34C979F3ABAA,1.622762e+12,111.0,116.0,bills,Claim,Claim 3,18,None,None,NaN,0.0
22116,0DBB894BB176,1.618593e+12,512.0,523.0,questioning,Claim,Claim 3,88,None,None,NaN,0.0
38875,EF17CE533FCB,1.617040e+12,444.0,451.0,details,Claim,Claim 3,68,None,None,NaN,0.0
43718,B65ABA471638,1.617294e+12,149.0,159.0,atmosphere,Claim,Claim 2,24,None,None,NaN,0.0
45257,E49069AE274F,1.616973e+12,208.0,215.0,studies,Claim,Claim 2,36,None,None,NaN,0.0
50405,51D38A8BF1A5,1.619487e+12,360.0,369.0,calculate,Claim,Claim 2,59,None,None,NaN,0.0


In [21]:
dfna = df.fillna(0)

In [24]:
dfna[dfna['center'].isna()]

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,box_center,box_length,center,length


In [25]:
dfna.to_csv("train.csv")